Collecting Data

In [41]:
import pandas as pd
import requests as r
import datetime

In [42]:
# modules and code to upload clean tables to DBeaver
import sql_functions as sf
import psycopg2

Consumer Price Inflation UK (1988-2023) <br>
[Source](https://www.ons.gov.uk/economy/inflationandpriceindices/datasets/consumerpriceinflation) <br>
with 2015=100

In [43]:
cpi_UK = pd.read_csv('/Users/lishausmann/NeueFische/Tap_vs_Bottle/data/Consumer_price_index_UK.csv',
                     skiprows = 6)
cpi_UK.head()

,index date,name,CPIH ALL ITEMS,FOOD AND NON-ALCOHOLIC BEVERAGES,FOOD,BREAD & CEREALS,MEAT,FISH,"MILK, CHEESE & EGGS",OILS & FATS,...,"Energy, Food, Alcohol & Tobacco",Energy & Non-processed Food,Energy & Seasonal Food,"Education, Health & Social Protection",CPIH excluding tobacco,Industrial Goods,Miscellaneous and Other Services,CPIH excluding owner occupiers' housing costs,CPIH excluding council tax,CPIH excluding owner occupiers' housing costs and council tax and rates
0,198801.0,Jan.-88,"46,851","47,901","48,244","50,053","53,741","43,321","48,488","45,461",...,"36,679","37,044","33,852","22,323","48,106","78,883","26,778","47,734","47,304","48,329"
1,198802.0,Feb.-88,"46,998","48,174","48,501","50,353","53,386","43,670","48,523","45,921",...,"36,761","37,084","33,968","22,340","48,259","79,037","26,915","47,884","47,455","48,486"
2,198803.0,März-88,"47,202","48,324","48,643","50,311","53,721","43,130","48,551","46,108",...,"36,831","37,144","33,976","22,455","48,476","79,399","26,968","48,042","47,667","48,652"
3,198804.0,Apr.-88,"47,801","48,592","48,894","50,548","53,736","43,151","48,861","46,249",...,"37,198","37,531","34,434","22,510","49,085","80,067","27,098","48,707","48,184","49,214"
4,198805.0,Mai-88,"48,044","48,714","48,968","50,970","54,186","42,799","48,770","46,346",...,"37,360","37,688","34,535","22,544","49,336","80,623","27,213","48,941","48,435","49,459"


In [44]:
#headers to naming convention
cpi_UK.columns = cpi_UK.columns.str.strip()
cpi_UK.columns = cpi_UK.columns.str.lower()
cpi_UK.columns = cpi_UK.columns.str.replace(' ','_')
cpi_UK.head()

,index_date,name,cpih_all_items,food_and_non-alcoholic_beverages,food,bread_&_cereals,meat,fish,"milk,_cheese_&_eggs",oils_&_fats,...,"energy,_food,_alcohol_&_tobacco",energy_&_non-processed_food,energy_&_seasonal_food,"education,_health_&_social_protection",cpih_excluding_tobacco,industrial_goods,miscellaneous_and_other_services,cpih_excluding_owner_occupiers'_housing_costs,cpih_excluding_council_tax,cpih_excluding_owner_occupiers'_housing_costs_and_council_tax_and_rates
0,198801.0,Jan.-88,"46,851","47,901","48,244","50,053","53,741","43,321","48,488","45,461",...,"36,679","37,044","33,852","22,323","48,106","78,883","26,778","47,734","47,304","48,329"
1,198802.0,Feb.-88,"46,998","48,174","48,501","50,353","53,386","43,670","48,523","45,921",...,"36,761","37,084","33,968","22,340","48,259","79,037","26,915","47,884","47,455","48,486"
2,198803.0,März-88,"47,202","48,324","48,643","50,311","53,721","43,130","48,551","46,108",...,"36,831","37,144","33,976","22,455","48,476","79,399","26,968","48,042","47,667","48,652"
3,198804.0,Apr.-88,"47,801","48,592","48,894","50,548","53,736","43,151","48,861","46,249",...,"37,198","37,531","34,434","22,510","49,085","80,067","27,098","48,707","48,184","49,214"
4,198805.0,Mai-88,"48,044","48,714","48,968","50,970","54,186","42,799","48,770","46,346",...,"37,360","37,688","34,535","22,544","49,336","80,623","27,213","48,941","48,435","49,459"


In [45]:
#drop rows with comments to the dataframe
cpi_UK.drop(cpi_UK.index[432:], axis=0, inplace = True)

In [46]:
# index_date to datetime
cpi_UK.index_date = cpi_UK.index_date.astype(int)
cpi_UK.index_date = cpi_UK.index_date.astype(str) + '01'
cpi_UK.index_date = pd.to_datetime(cpi_UK.index_date.astype(int), format = '%Y%m%d')


In [47]:
#confirmation datatype of values in df is string as suspected
type(cpi_UK.food[1])

str

In [48]:
cpi_UK= cpi_UK.replace(',','.', regex=True)
cpi_UK.head()

,index_date,name,cpih_all_items,food_and_non-alcoholic_beverages,food,bread_&_cereals,meat,fish,"milk,_cheese_&_eggs",oils_&_fats,...,"energy,_food,_alcohol_&_tobacco",energy_&_non-processed_food,energy_&_seasonal_food,"education,_health_&_social_protection",cpih_excluding_tobacco,industrial_goods,miscellaneous_and_other_services,cpih_excluding_owner_occupiers'_housing_costs,cpih_excluding_council_tax,cpih_excluding_owner_occupiers'_housing_costs_and_council_tax_and_rates
0,1988-01-01,Jan.-88,46.851,47.901,48.244,50.053,53.741,43.321,48.488,45.461,...,36.679,37.044,33.852,22.323,48.106,78.883,26.778,47.734,47.304,48.329
1,1988-02-01,Feb.-88,46.998,48.174,48.501,50.353,53.386,43.670,48.523,45.921,...,36.761,37.084,33.968,22.340,48.259,79.037,26.915,47.884,47.455,48.486
2,1988-03-01,März-88,47.202,48.324,48.643,50.311,53.721,43.130,48.551,46.108,...,36.831,37.144,33.976,22.455,48.476,79.399,26.968,48.042,47.667,48.652
3,1988-04-01,Apr.-88,47.801,48.592,48.894,50.548,53.736,43.151,48.861,46.249,...,37.198,37.531,34.434,22.510,49.085,80.067,27.098,48.707,48.184,49.214
4,1988-05-01,Mai-88,48.044,48.714,48.968,50.970,54.186,42.799,48.770,46.346,...,37.360,37.688,34.535,22.544,49.336,80.623,27.213,48.941,48.435,49.459


In [49]:
cpi_UK= cpi_UK.replace('..',0)
cpi_UK.head()

,index_date,name,cpih_all_items,food_and_non-alcoholic_beverages,food,bread_&_cereals,meat,fish,"milk,_cheese_&_eggs",oils_&_fats,...,"energy,_food,_alcohol_&_tobacco",energy_&_non-processed_food,energy_&_seasonal_food,"education,_health_&_social_protection",cpih_excluding_tobacco,industrial_goods,miscellaneous_and_other_services,cpih_excluding_owner_occupiers'_housing_costs,cpih_excluding_council_tax,cpih_excluding_owner_occupiers'_housing_costs_and_council_tax_and_rates
0,1988-01-01,Jan.-88,46.851,47.901,48.244,50.053,53.741,43.321,48.488,45.461,...,36.679,37.044,33.852,22.323,48.106,78.883,26.778,47.734,47.304,48.329
1,1988-02-01,Feb.-88,46.998,48.174,48.501,50.353,53.386,43.670,48.523,45.921,...,36.761,37.084,33.968,22.340,48.259,79.037,26.915,47.884,47.455,48.486
2,1988-03-01,März-88,47.202,48.324,48.643,50.311,53.721,43.130,48.551,46.108,...,36.831,37.144,33.976,22.455,48.476,79.399,26.968,48.042,47.667,48.652
3,1988-04-01,Apr.-88,47.801,48.592,48.894,50.548,53.736,43.151,48.861,46.249,...,37.198,37.531,34.434,22.510,49.085,80.067,27.098,48.707,48.184,49.214
4,1988-05-01,Mai-88,48.044,48.714,48.968,50.970,54.186,42.799,48.770,46.346,...,37.360,37.688,34.535,22.544,49.336,80.623,27.213,48.941,48.435,49.459


In [50]:
cpi_UK.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432 entries, 0 to 431
Columns: 181 entries, index_date to cpih_excluding_owner_occupiers'_housing_costs_and_council_tax_and_rates
dtypes: datetime64[ns](1), float64(1), object(179)
memory usage: 611.0+ KB


In [51]:
# convert chosen headers to naming convention
var = 'index_date, cpih_all_items, FOOD AND NON-ALCOHOLIC BEVERAGES, FOOD, NON-ALCOHOLIC BEVERAGES, MINERAL WATERS,_SOFT DRINKS AND JUICES, ALCOHOLIC BEVERAGES, CLOTHING AND FOOTWEAR, WATER SUPPLY, ELECTRICITY, GAS, HEALTH, PHARMACEUTICAL PRODUCTS, PACKAGE HOLIDAY, HOUSING,_WATER,_ELELCTRICITY,_GAS AND OTHER FUELS'
columns_cpi = var.split(', ')
columns_cpi = list(map(lambda x: x.lower(), columns_cpi))
columns_cpi = list(map(lambda x: x.replace(' ', '_'), columns_cpi))
columns_cpi

['index_date',
 'cpih_all_items',
 'food_and_non-alcoholic_beverages',
 'food',
 'non-alcoholic_beverages',
 'mineral_waters,_soft_drinks_and_juices',
 'alcoholic_beverages',
 'clothing_and_footwear',
 'water_supply',
 'electricity',
 'gas',
 'health',
 'pharmaceutical_products',
 'package_holiday',
 'housing,_water,_elelctricity,_gas_and_other_fuels']

In [60]:
# create subset with chosen columns of the original df
cpi_UK_water = cpi_UK[columns_cpi]

In [68]:
cpi_UK_water[range(1,15)] = cpi_UK_water[range(1,15)].astype(float)

KeyError: "None of [Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], dtype='int64')] are in the [columns]"

In [67]:
cpi_UK_water.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432 entries, 0 to 431
Data columns (total 15 columns):
 #   Column                                             Non-Null Count  Dtype         
---  ------                                             --------------  -----         
 0   index_date                                         432 non-null    datetime64[ns]
 1   cpih_all_items                                     432 non-null    float64       
 2   food_and_non-alcoholic_beverages                   432 non-null    float64       
 3   food                                               432 non-null    float64       
 4   non-alcoholic_beverages                            432 non-null    float64       
 5   mineral_waters,_soft_drinks_and_juices             432 non-null    float64       
 6   alcoholic_beverages                                432 non-null    float64       
 7   clothing_and_footwear                              432 non-null    float64       
 8   water_supply        

In [69]:
sf.upload(cpi_UK_water, 'cpi UK sub')

The cpi_uk_sub table was imported successfully.
